In [1]:
# print ('running part 2')
# print ()
# !python part2.py

# print ()

print ('running part 3')
print ()
!python part3.py

# print ()


print ('running part 4')
print ()
!python part4.py

# my_dict = {'hello':0, 'my':1, 'name':2, 'is:':3, 'Reuben':4}

# # print (max(my_dict.values()))

# for j in range(len(my_dict)-1, -1, -1):
#     print (my_dict[list(my_dict.keys())[j]])

running part 3

============================ EN ============================
time to get predictions for EN :  0.32213807106018066

EN
Entities in gold data : 802
Entities in prediction: 701

entities
 - Correct   : 506
 - Precision : 0.7218
 - Recall    : 0.6309
 - F score   : 0.6733

entity types
 - Correct   : 476
 - Precision : 0.6790
 - Recall    : 0.5935
 - F score   : 0.6334

============================ FR ============================
time to get predictions for FR :  0.6502947807312012

FR
Entities in gold data : 238
Entities in prediction: 173

entities
 - Correct   : 126
 - Precision : 0.7283
 - Recall    : 0.5294
 - F score   : 0.6131

entity types
 - Correct   : 80
 - Precision : 0.4624
 - Recall    : 0.3361
 - F score   : 0.3893

============================ Predictions Complete ============================
running part 4

============================ EN ============================
time to get predictions for EN :  2.041571855545044

EN
Entities in gold data : 802
Entiti